In [1]:
from collections import defaultdict
import numpy as np
import sys
from enum import Enum
class ACTION(Enum):
    high=1
    low=0 
    
class Winner:
    def __init__(self):
        self.P_H = 0.55
        self.P_L = 0.45
        self.state = 0
        self.done = 0
        self.R = 0
        self.highcost = -50
        self.lowcost = -10
        self.greedy = 0.1
        self.discount=0.9
        self.init_Q()
        self.alpha=0.0001
        self.lam = 0.9
    def reset_game(self):
        self.state=0
        self.done=0
    def ba_policy(self):
        sample = np.random.uniform(0, 1)
        if sample > 0.5: 
            ac = ACTION.low
        else:
            ac = ACTION.high
        return ac
        
    #def ta_policy(self):
    def TDlam(self,numepisode):
        weight=self.init_weight()
        for i_episode in range(1, numepisode+1):
            if i_episode % 1000 == 0:
                print("\rEpisode {}/{}.".format(i_episode, numepisode), end="")
                sys.stdout.flush()
            
            state = 0
            x = self.fea_vector(state)
            z = np.zeros_like(x)
            v_old = 0
            while(1):
                action = self.ba_policy()
                
                reward,next_state,done = self.env_step(state,action)
                
                x_next = self.fea_vector(next_state)
                
                v = np.dot(weight,x)
                
                v_next = np.dot(weight,x_next)
                
                delta = reward + self.discount * v_next - v
                
                z = self.discount*self.lam*z + (1 - self.alpha*self.discount*self.lam*np.dot(z,x))*x
                
                weight = weight + self.alpha*(delta + v - v_old)*z - self.alpha * (v - v_old)*x
                
                v_old = v_next
                
                x = x_next
                
                state = next_state
                
                if done == 1 :
                    break

        return weight
    def init_weight(self):
        
        w = np.zeros((7))
        return w
    def fea_vector(self,state):
        vec = np.zeros((7))
        vec[state+3] = 1
        return vec
        
        
        
    def env_step(self,state,action):
        done=0
        reward=0
        sample = np.random.uniform(0, 1)
        if action == ACTION.high:
            reward = reward + self.highcost
            suc_prob = self.P_H
        else:
            suc_prob = self.P_L
            reward = reward + self.lowcost
        if sample < suc_prob:
            state = state + 1
        else:
            state = state -1
        if state >= 3 :
            reward = reward + 1000
            done = 1
        if state <= (-3):
            done = 1
        return reward,state,done
    def init_Q(self):
        self.Q = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.Q[state-3,act]=0.0
    def init_C(self):
        
        self.C = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.C[state-3,act]=0.0
        

In [4]:
winner=Winner()

w=winner.TDlam(500000)
for i in range(7):
    
    print(np.dot (winner.fea_vector(i-3),w))

Episode 500000/500000.0.0
-30.074500878126027
-0.9753338374196885
94.94075177696503
279.8884506882318
592.7950723778574
0.0


In [5]:
for i in range(7):
    
    print(np.dot (winner.fea_vector(i-3),w))

0.0
-30.074500878126027
-0.9753338374196885
94.94075177696503
279.8884506882318
592.7950723778574
0.0
